<img src="https://duckdb.org/images/DuckDB_Logo_dl.png" width="150">

__Package Install__

```pip install duckdb==0.8.1```

In [1]:
import duckdb
import pandas as pd

## 👉 Connect DuckDB
* Connect in Memory ```duckdb.connect("")```
* Connect to files ```duckdb.connect("db.db")```

In [3]:
mydb = duckdb.connect(database=':memory:', read_only=False)

## 👉 Read CSV to Pandas

---

__Multiple files__
```sql
-- read all files with a name ending in ".csv" in the folder "dir"
SELECT * FROM 'dir/*.csv';
-- read all files with a name ending in ".csv", two directories deep
SELECT * FROM '*/*/*.csv';
-- read all files with a name ending in ".csv", at any depth in the folder "dir"
SELECT * FROM 'dir/**/*.csv';
-- read the CSV files 'flights1.csv' and 'flights2.csv'
SELECT * FROM read_csv_auto(['flights1.csv', 'flights2.csv'])
-- read the CSV files 'flights1.csv' and 'flights2.csv', unifying schemas by name and outputting a `filename` column
SELECT * FROM read_csv_auto(['flights1.csv', 'flights2.csv'], union_by_name=True, filename=True)
```
__Parquet files__
```sql
-- read all files that match the glob pattern
SELECT * FROM 'test/*.parquet';
-- read 3 parquet files and treat them as a single table
SELECT * FROM read_parquet(['file1.parquet', 'file2.parquet', 'file3.parquet']);
-- Read all parquet files from 2 specific folders
SELECT * FROM read_parquet(['folder1/*.parquet','folder2/*.parquet']);
-- read all parquet files that match the glob pattern at any depth
SELECT * FROM read_parquet('dir/**/*.parquet');

```
### Read From URL run this Command 
```py
mydb.sql("INSTALL httpfs;")
mydb.sql("LOAD httpfs;")
```

In [ ]:
mydata = duckdb.execute("""SELECT * FROM read_csv_auto('engineer.csv');""").df()

## 👉 Read Parquet
---
__Multiple files__
```sql
-- read data from a hive partitioned data set
SELECT * FROM parquet_scan('orders/*/*/*.parquet', hive_partitioning=1);
-- write a table to a hive partitioned data set
COPY orders TO 'orders' (FORMAT PARQUET, PARTITION_BY (year, month));
```

In [ ]:
mydata = duckdb.execute("""SELECT * FROM parquet_scan('engineer.parquet');""").df()

## 👉 Copy Export Data to Another files

---
__Copy__
```sql
-- read a CSV file into the lineitem table - using auto-detected options
COPY lineitem FROM 'lineitem.csv' (AUTO_DETECT TRUE);
-- read a parquet file into the lineitem table
COPY lineitem FROM 'lineitem.pq' (FORMAT PARQUET);
-- read a json file into the lineitem table - using auto-detected options
COPY lineitem FROM 'lineitem.json' (FORMAT JSON, AUTO_DETECT TRUE);

-- write a table to a CSV file
COPY lineitem TO 'lineitem.csv' (FORMAT CSV, DELIMITER '|', HEADER);
-- write the result of a query to a Parquet file
COPY (SELECT l_orderkey, l_partkey FROM lineitem) TO 'lineitem.parquet' (COMPRESSION ZSTD);
```
__Copy From__
```sql
-- Copy the contents of a comma-separated file 'test.csv' without a header into the table 'test'
COPY test FROM 'test.csv';
-- Copy the contents of a comma-separated file with a header into the 'category' table
COPY category FROM 'categories.csv' ( HEADER );
-- Copy the contents of 'lineitem.tbl' into the 'lineitem' table, where the contents are delimited by a pipe character ('|')
COPY lineitem FROM 'lineitem.tbl' ( DELIMITER '|' );
-- Copy the contents of 'lineitem.tbl' into the 'lineitem' table, where the delimiter, quote character, and presence of a header are automatically detected
COPY lineitem FROM 'lineitem.tbl' ( AUTO_DETECT TRUE );
-- Read the contents of a comma-separated file 'names.csv' into the 'name' column of the 'category' table. Any other columns of this table are filled with their default value.
COPY category(name) FROM 'names.csv';
-- Read the contents of a parquet file 'lineitem.parquet' into the lineitem table
COPY lineitem FROM 'lineitem.parquet' ( FORMAT PARQUET );
-- Read the contents of a newline-delimited json file 'lineitem.ndjson' into the lineitem table
COPY lineitem FROM 'lineitem.ndjson' ( FORMAT JSON );
-- Read the contents of a json file 'lineitem.json' into the lineitem table
COPY lineitem FROM 'lineitem.json' ( FORMAT JSON, ARRAY TRUE );
```
__Copy to__
```sql
-- Copy the contents of the 'lineitem' table to the file 'lineitem.tbl', where the columns are delimited by a pipe character ('|'), including a header line.
COPY lineitem TO 'lineitem.tbl' ( DELIMITER '|', HEADER );
-- Copy the l_orderkey column of the 'lineitem' table to the file 'orderkey.tbl'
COPY lineitem(l_orderkey) TO 'orderkey.tbl' ( DELIMITER '|' );
-- Copy the result of a query to the file 'query.csv', including a header with column names
COPY (SELECT 42 AS a, 'hello' AS b) TO 'query.csv' WITH (HEADER 1, DELIMITER ',');
-- Copy the result of a query to the Parquet file 'query.parquet'
COPY (SELECT 42 AS a, 'hello' AS b) TO 'query.parquet' (FORMAT PARQUET);
-- Copy the result of a query to the newline-delimited JSON file 'query.ndjson'
COPY (SELECT 42 AS a, 'hello' AS b) TO 'query.ndjson' (FORMAT JSON);
-- Copy the result of a query to the JSON file 'query.json'
COPY (SELECT 42 AS a, 'hello' AS b) TO 'query.json' (FORMAT JSON, ARRAY TRUE);
```

## 👉 SQL to Pandas

In [ ]:
my_df = pandas.DataFrame.from_dict({'a': [42]})
results = duckdb.sql("SELECT * FROM my_df").df()

## 👉 Import From Pandas

In [ ]:
my_df = pandas.DataFrame.from_dict({'a': [42]})
duckdb.sql("CREATE TABLE my_table AS SELECT * FROM my_df")
duckdb.sql("INSERT INTO my_table SELECT * FROM my_df")

## 👉 Create Table
```sql
CREATE TABLE weather (
    city           VARCHAR,
    temp_lo        INTEGER, -- minimum temperature on a day
    temp_hi        INTEGER, -- maximum temperature on a day
    prcp           REAL,
    date           DATE
);
```

## 👉 Export MemoryDB to FileDB
---
__More command__
```sql
-- export the table contents with the given options
EXPORT DATABASE 'target_directory' (FORMAT CSV, DELIMITER '|');
-- export the table contents as parquet
EXPORT DATABASE 'target_directory' (FORMAT PARQUET);
-- export as parquet, compressed with ZSTD, with a row_group_size of 100000
EXPORT DATABASE 'target_directory' (FORMAT PARQUET, COMPRESSION ZSTD, ROW_GROUP_SIZE 100000);
```

In [ ]:
duckdb.execute("EXPORT DATABASE './nook.db';")

## 👉 Import FileDB to MemoryDB

In [ ]:
duckdb.execute("IMPORT DATABASE 'target_directory'")

## 👉 Create view to ETL or Cleansing
---
__More command__

```sql
-- create a simple view
CREATE VIEW v1 AS SELECT * FROM tbl;
-- create a view or replace it if a view with that name already exists
CREATE OR REPLACE VIEW v1 AS SELECT 42;
-- create a view and replace the column names
CREATE VIEW v1(a) AS SELECT 42;
```

## 👉 Rebuild DB

In [ ]:
duckdb.execute("VACUUM ANALYZE;")